In [17]:
import pandas as pd
import numpy as np
import os.path
import recsys as rs
import notipy
import math
import sys
import TopSimilarRecommender as TSR
import ItemBasedRecommender as IBR
import random
from scipy import sparse as sps
import importlib

In [18]:
train = pd.read_csv('Data/train_final.csv','\t')
tracks = pd.read_csv('Data/tracks_final.csv','\t')
pl_info = pd.read_csv('Data/playlists_final.csv','\t')
tgt_playlists = pd.read_csv('Data/target_playlists.csv','\t')
tgt_tracks = pd.read_csv('Data/target_tracks.csv','\t')

In [19]:
train, test, tgt_tracks, tgt_playlists = rs.split_train_test(train, 10, 20, 5, 2517)
#tgt_playlists.shape

In [20]:
ibr = IBR.ItemBasedRecommender(idf=True, shrinkage=10,n_el_sim=20)
ibr.fit(tracks,train,tgt_tracks)
print('Model fitted!')

  0%|          | 18/4420 [00:00<00:25, 175.13it/s]

Calculated Indices


  0%|          | 0/100000 [00:00<?, ?it/s]

(4420, 100000)
Model URM built
Model URM regularized with IDF!


100%|██████████| 100000/100000 [11:21<00:00, 146.80it/s]

Similarity built
(100000, 17185)
Model fitted!


In [21]:
tsr= TSR.TopSimilarRecommender(attributes=['artist_id', 'album', 'playcount'],idf=True, n_min_attr=90, n_el_sim=20)
tsr.fit(tracks,tgt_tracks )
print('Model fitted!')

Fixed dataset
Calculated Indices


  0%|          | 54/100000 [00:00<03:05, 539.33it/s]

ICM built
ICM regularized with IDF!


100%|██████████| 100000/100000 [12:10<00:00, 136.80it/s]

Similarity built
Model fitted!


In [22]:
rs = importlib.reload(rs)
similarity_matrix = rs. merge_similarities(tsr.S, ibr.S, 0.7)

In [23]:
similarity_matrix

<100000x17185 sparse matrix of type '<class 'numpy.float64'>'
	with 1627862 stored elements in Compressed Sparse Row format>

# Recommendation

In [24]:
_, _, IX_tgt_playlists, _ = rs.create_sparse_indexes(playlists=tgt_playlists)
URM = rs.create_tgt_URM(IX_tgt_playlists, tsr.IX_items, train)
URM = URM.tocsr()
print('URM built')

100%|██████████| 4420/4420 [00:21<00:00, 204.70it/s]

URM built


In [59]:
rs = importlib.reload(rs)
recommendetions = np.array([])
div = similarity_matrix.sum(axis=0)
for p in IX_tgt_playlists.values:
    avg_sims = URM[p,:].dot(similarity_matrix).toarray().ravel()
    avg_sims = np.divide(avg_sims,div+12000) #VERY LOW RESULT
    avg_sims =np.squeeze(np.asarray(avg_sims))
    top = rs.top5_outside_playlist(avg_sims, p, train, IX_tgt_playlists, tsr.IX_tgt_items, False, False)
    recommendetions = np.append(recommendetions, rs.sub_format(top))
    if (p % 1000 == 0):
        print('Recommended ' + str(p) + ' users over ' + str(IX_tgt_playlists.values.shape[0]))

rec_df =  pd.DataFrame({'playlist_id' : IX_tgt_playlists.index.values, 'track_ids' : recommendetions})

Recommended 0 users over 4420
Recommended 1000 users over 4420
Recommended 2000 users over 4420
Recommended 3000 users over 4420
Recommended 4000 users over 4420


# Evaluation

In [60]:
def evaluate(results, test, eval_metric='MAP'):
    if eval_metric == 'MAP':
        APs = results.apply(calculate_AP, axis=1, args=(test,))
        res = (APs.sum())/results.shape[0]
    return res

def calculate_AP(row, test):
    p_id = row['playlist_id']
    recs = np.fromstring(row['track_ids'], dtype=float, sep=' ')

    AP = 0
    rel_sum = 0
    n_rel_items = min(test[test['playlist_id'] == p_id].shape[0],5)
    for i in range(recs.size):
        rel = 1 if ((test['playlist_id'] == int(p_id)) & (test['track_id'] == recs[i])).any() else 0
        rel_sum += rel
        P = rel_sum/(i+1)
        AP += (P * rel)/n_rel_items

    return AP

In [61]:
map_eval = evaluate(rec_df, test, 'MAP')
print('Evaluation completed!')

Evaluation completed!


In [62]:
map_eval

0.0944170437405732

# Save to csv

In [63]:
rec_df.to_csv('Submissions/ensemble_submission_notags_alpha07_nel65,shr=2_idf=T' + str(0) + '.csv', index=False)
print('Results saved as csv!')

Results saved as csv!


In [54]:
#0.09471794871794867
#con 1000 0.093
#con 10000 0.09435595776772246
